# 🎙️ 自动字幕
日语直播音频 → 中日双语 SRT 字幕

**运行前确认**：菜单栏 → 修改 → 笔记本设置 → 硬件加速器选T4 GPU

依次运行每个格子即可。

In [ ]:
# ① 克隆仓库 & 安装依赖
!git clone https://github.com/ushi-C/jpVTuberlivesrtja-zh-allAItool.git
%cd jpVTuberlivesrtja-zh-allAItool
!pip install -r requirements.txt -q
!nvidia-smi

In [ ]:
# ② 导入模块 & 初始化
import sys
sys.path.insert(0, '/content/jpVTuberlivesrtja-zh-allAItool')

from pathlib import Path
from google.colab import files
from api_client import init_openai_client, usage_stats
from asr import run_asr
from danmaku_clean import run_danmaku_cleaning
from proofread import run_smart_proofread
from translate import run_parallel_translation
from utils import format_srt_time
from typing import List, Tuple, Dict

client = init_openai_client()

In [ ]:
# ③ 输入直播背景信息
print('=== 直播背景信息 ===')
host_info = input('1. 请输入【配信者/所属/ファンネーム】(例如：ななしいんく所属 涼海ネモ): ').strip()
stream_title = input('2. 请输入【直播标题】(例如：雑談): ').strip()

my_bg = {
    'host_info': host_info if host_info else '未提供',
    'stream_title': stream_title if stream_title else '通用直播',
}

In [ ]:
# ④ 上传音频文件（必须）
print('📤 上传音频文件（mp3 / m4a / wav / mp4）:')
uploaded = files.upload()
audio_path = list(uploaded.keys())[0]
print(f'✅ 已接收：{audio_path}')

In [ ]:
# ⑤ 是否上传弹幕 JSON
print('📤 是否上传弹幕 JSON？(yes/no)')
use_dm = input().strip().lower()

dm_path = None

if use_dm == 'yes':
    print('📤 上传弹幕 JSON 文件:')
    uploaded_dm = files.upload()
    if uploaded_dm:
        dm_path = list(uploaded_dm.keys())[0]
        print(f'✅ 已接收：{dm_path}')
    else:
        print('⚠️ 未检测到文件，将跳过校对')
else:
    print('⏭️ 已跳过弹幕上传')

In [ ]:
# ⑥ 运行

# Step 1: ASR
raw_asr = run_asr(audio_path)

# Step 2: 弹幕清洗
clean_dm = []
if dm_path:
    clean_dm = run_danmaku_cleaning(dm_path)
else:
    print('🛈 无弹幕数据，进入0校模式')

# Step 3: 智能校对
proofed_data = run_smart_proofread(client, raw_asr, clean_dm, my_bg)

# Step 4: 翻译
final_data = run_parallel_translation(client, proofed_data)

# Step 5: 写 SRT
srt_file = f"{Path(audio_path).stem}_bilingual.srt"
with open(srt_file, 'w', encoding='utf-8') as f:
    for i, s in enumerate(final_data, 1):
        f.write(
            f"{i}\n"
            f"{format_srt_time(s['start'])} --> {format_srt_time(s['end'])}\n"
            f"{s['ja']}\n{s['zh']}\n\n"
        )

print(f'\n✅ 任务结束。Token 消耗估算: {usage_stats.total_tokens}')
files.download(srt_file)